### **Preparação do ambiente para executar Spark no Google Colab**

In [5]:
# Configurar o Google Colab para execução do Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz  
!pip install -q findspark
!pip install -q pyspark

     |████████████████████████████████| 215.7MB 1.2MB/s 
     |████████████████████████████████| 204kB 60.1MB/s 


In [0]:
# Atribuir variáveis de ambiente para o Java e o Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
# Importar o FindSpark para localizar a instalação do spark e iniciar uma sessão
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
import pyspark.sql.functions as f

In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [28]:
ls -la gdrive/'My Drive'/'Desafio Técnico - Semantix'


total 364313
-rw------- 1 root root 167813770 Oct  2 11:37 NASA_access_log_Aug95
-rw------- 1 root root 205242368 Oct  2 11:40 NASA_access_log_Jul95


In [0]:
#dsAug = spark.read.text("gdrive/My Drive/Desafio Técnico - Semantix/NASA_access_log_Aug95", lineSep=" ")
dfAug = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "false") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .load("gdrive/My Drive/Desafio Técnico - Semantix/NASA_access_log_Aug95")


In [0]:
dfJul = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "false") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .load("gdrive/My Drive/Desafio Técnico - Semantix/NASA_access_log_Jul95")

In [0]:
df = dfJul.union(dfAug)

In [0]:
df.createTempView("tbl_logs")


In [0]:
df2 = spark.sql("SELECT _c0 as Host, _c3 as Data, _c5 as Urls,_c6 as retorno, _c7 as byte FROM tbl_logs")

In [0]:
qtde_hosts_unicos = df2.select("Host").distinct().count()
#df.select("some_column").distinct.count

## **1. Número de hosts únicos.**

In [87]:
f"Hosts distintos: {qtde_hosts_unicos}"

'Hosts distintos: 137979'

## **2. O total de erros 404**

In [93]:
df2.where(df2['retorno'] == "404").count()

20871

## **3. Os 5 URLs que mais causaram erro 404.**

In [95]:
df2.where(df2['retorno'] == "404").groupBy("Host","retorno").count().orderBy("count", ascending=False).show(5)

+--------------------+-------+-----+
|                Host|retorno|count|
+--------------------+-------+-----+
|hoohoo.ncsa.uiuc.edu|    404|  251|
|piweba3y.prodigy.com|    404|  156|
|jbiagioni.npt.nuw...|    404|  132|
|piweba1y.prodigy.com|    404|  114|
|www-d4.proxy.aol.com|    404|   91|
+--------------------+-------+-----+
only showing top 5 rows



## **5. O total de bytes retornados.**

In [104]:
df2.groupBy().agg(f.sum('byte')).show()

#df.groupBy().agg(F.sum('Number')).show()

+---------------+
|      sum(byte)|
+---------------+
|6.5524319796E10|
+---------------+

